<a href="https://colab.research.google.com/github/Heffx/CNN/blob/master/Projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Importando o Pytorch
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [2]:
# Conferindo se o CUDA esta disponivel

verificaGPU = torch.cuda.is_available()

if not verificaGPU:
  print('CUDA INDISPONIVEL, TREINANDO COM CPU')
else:
  print('CUDA DISPONIVEL, TREINANDO COM GPU')

CUDA DISPONIVEL, TREINANDO COM GPU


In [3]:
# Instalando o Torch Vision
!pip3 install torchvision

In [4]:
# Instalando Pillow
!pip install Pillow==4.1.1

!pip install image

In [0]:
# Fazendo download e descompactando o dataset
!wget -cq https://github.com/udacity/pytorch_challenge/raw/master/cat_to_name.json
!wget -cq https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip
!rm -r flower_data || true
!unzip -qq flower_data.zip

In [0]:
# Todos os imports necessários
import time
import json
import copy
import flower_data
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import PIL
from PIL import Image
from collections import OrderedDict
import torch
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F
import glob
import os
from skimage import data, io, filters
from scipy.optimize import minimize


In [0]:
# Definindo variavel global para treinamento com CUDA
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#Carregando, Normalizando e Criando vetor de Imagens

In [0]:
# Aumento e normalização de dados para treinamento
# Apenas normalização para validação
data_transforms = {
    'train': transforms.Compose([
        # Redimensiona a imagem em (224,224)
        transforms.RandomResizedCrop(224),
        # Inverte horizontalmente a imagem
        transforms.RandomHorizontalFlip(),
        # Transforma em matriz de dados unicos
        transforms.ToTensor(),
        # Normaliza as imagens atraves de desvio padrão(Valores retirados de outro trabalho) 
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        # Transformação da imagem para tamanho de entrada
        transforms.Resize(256),
        # Recorta a imagem no centro
        transforms.CenterCrop(224),
        # Transforma em matriz de dados unicos
        transforms.ToTensor(),
        # Normaliza as imagens atraves de desvio padrão(Valores retirados de outro trabalho) 
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


# print(data_transforms)

In [0]:
# Definindo nome dos diretorios de forma global para todo o código
train_dir = 'flower_data/train/'
valid_dir = 'flower_data/valid/'
dirs = {'train': train_dir, 
        'valid': valid_dir, 
       }

In [0]:
# Carregando as pastas do dataset
image_datasets = {x: datasets.ImageFolder(dirs[x],
                                          transform = data_transforms[x])
                  for x in ['train', 'valid']}

# Carregando imagens do dataset de treino e validação em  lotes
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], 
                                              batch_size=32, shuffle=True) 
               for x in ['train', 'valid']}

# Carregando tamanho do dataset de treino e validação
dataset_sizes = {x: len(image_datasets[x]) 
                              for x in ['train', 'valid']}

# Carregando as classes do dataset de treino
class_names = image_datasets['train'].classes

In [0]:
# print(dataset_sizes)
# print(device)

In [0]:
# Carregando arquivo Json
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [13]:
# Atribuindo o nome das classes com o arquivo Json
images, labels = next(iter(dataloaders['train']))
images.size()
images, labels = next(iter(dataloaders['train']))
rand_idx = np.random.randint(len(images))
# print(rand_idx)
print("label: {}, class: {}, name: {}".format(labels[rand_idx].item(),
                                               class_names[labels[rand_idx].item()],
                                               cat_to_name[class_names[labels[rand_idx].item()]]))

label: 72, class: 72, name: azalea


## Pre-treino

In [0]:
# Carregando em model a rede VGG19
model = models.vgg19(pretrained=True)

In [15]:
# Verificando se o modelo foi carregado corretamente
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inplace)

In [0]:
#Definindo a arquitetura da rede
class Classifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(25088,4096)
    self.fc2 = nn.Linear(4096,102)
    
  def forward(self,x):
    # Verifica se o tensor está "flattened (achatado)"
    x = x.view(x.shape[0],-1)
    
    x = F.relu(self.fc1(x))
    x = F.log_softmax(self.fc2(x),dim=1)
    
    return x

In [0]:
classifier = Classifier()
for param in model.parameters():
    param.requires_grad = False
    
model.classifier = classifier

In [0]:
if verificaGPU:
    model.cuda()

In [0]:
# Criteria NLLLoss which is recommended with Softmax final layer
criterion = nn.NLLLoss()
# Observe that all parameters are being optimized
optim = torch.optim.Adam(model.classifier.parameters(), lr=0.001)
# Decay LR by a factor of 0.1 every 4 epochs
sched = lr_scheduler.StepLR(optim, step_size=4, gamma=0.1)
# Number of epochs
eps=10

In [0]:
def train_model(model, criteria, optimizer, scheduler,    
                                      num_epochs=25, device='cuda'):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Época {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                        
           # Acuracia
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            # Acc acuracia
            print('{} Loss: {:.4f} Acurácia: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Treino completo em {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Melhor Acurácia: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [21]:
model_ft = train_model(model, criterion, optim, sched, eps, 'cuda')

Época 0/9
----------
train Loss: 2.3995 Acurácia: 0.4913
valid Loss: 0.9234 Acurácia: 0.7396

Época 1/9
----------
train Loss: 0.9916 Acurácia: 0.7291
valid Loss: 0.6414 Acurácia: 0.8215

Época 2/9
----------
train Loss: 0.8424 Acurácia: 0.7688
valid Loss: 0.6553 Acurácia: 0.8056

Época 3/9
----------
train Loss: 0.7243 Acurácia: 0.8010
valid Loss: 0.5675 Acurácia: 0.8325

Época 4/9
----------
train Loss: 0.4972 Acurácia: 0.8631
valid Loss: 0.4016 Acurácia: 0.8912

Época 5/9
----------
train Loss: 0.4374 Acurácia: 0.8774
valid Loss: 0.3690 Acurácia: 0.9022

Época 6/9
----------
train Loss: 0.3816 Acurácia: 0.8922
valid Loss: 0.3560 Acurácia: 0.8998

Época 7/9
----------
train Loss: 0.3583 Acurácia: 0.8982
valid Loss: 0.3551 Acurácia: 0.9120

Época 8/9
----------
train Loss: 0.3460 Acurácia: 0.9006
valid Loss: 0.3495 Acurácia: 0.9181

Época 9/9
----------
train Loss: 0.3389 Acurácia: 0.9029
valid Loss: 0.3473 Acurácia: 0.9156

Treino completo em 35m 23s
Melhor Acurácia: 0.918093


In [0]:
# Salvando Modelo
torch.save(model.state_dict(), './vgg19.pth')

In [24]:
model_alex = models.alexnet(pretrained=True)

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.torch/models/alexnet-owt-4df8aa71.pth
100%|██████████| 244418560/244418560 [00:08<00:00, 28464971.35it/s]


In [25]:
model_alex

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p=0.5)
    (4): Linear(in_features=4096, out_feature

In [0]:
#Definindo a arquitetura da rede
class Classifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(512,256)
    self.fc2 = nn.Linear(256,12)
    
  def forward(self,x):
    # Verifica se o tensor está "flattened (achatado)"
    x = x.view(x.shape[0],-1)
    
    x = F.relu(self.fc1(x))
    x = F.log_softmax(self.fc2(x),dim=1)
    
    return x

In [0]:
classifier = Classifier()
for param in model_alex.parameters():
    param.requires_grad = False
    
model_alex.classifier = classifier

In [0]:
if verificaGPU:
    model_alex.cuda()

In [35]:
model_ft = train_model(model_alex, criterion, optim, sched, eps, 'cuda')

Época 0/9
----------


RuntimeError: ignored

In [0]:
# Salvando Modelo
torch.save(model_alex.state_dict(), './alexnet.pth')

In [0]:
# Carregando uma única imagem
image_path = 'flower_data/valid/29/image_04097.jpg'
img = Image.open(image_path)